In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
pd.set_option("display.max_rows",None)

In [3]:
wc_data = pd.read_csv('/Users/guest/Dropbox/Education/Thinkful/Unit 1/Lesson 3 (Intermediate Data Viz)/s6/WELLCOME_APCspend2013_forThinkful.csv', 
                      encoding = "ISO-8859-1")
list(wc_data)

['PMID/PMCID',
 'Publisher',
 'Journal title',
 'Article title',
 'COST (£) charged to Wellcome (inc VAT when charged)']

In [4]:
# Remove characters except for numbers, spaces, and decimals
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: re.sub('[^\.0-9 ]','', str(x))) 

# Remove trailing whitespace
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: str(x).strip())

# Separate 15 character strings to account for PMIDS and PMCIDS that don't have space between them
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: re.sub(r'(..)', r' ', x, 8) if len(x) == 15 else str(x))

# Add a blank placeholder for PMID to entries that only have PMCID
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: '99999 ' + x if len(str(x)) == 7 else x)

# Add a blank placeholder for PMCID to entries that only have PMID
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: x + ' 99999' if len(str(x)) == 8 else x)

# Replace invalid entries (length of < 13) with two blank placeholders
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: '99999 99999' if len(str(x)) < 13 else x)

# Split strings into lists on space
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: str(x).strip().split())

# Replace entries that are too long (>2) with two blank placeholders
wc_data['PMID/PMCID'] = wc_data['PMID/PMCID'].apply(lambda x: ['99999', '99999'] if len(x) > 2 else x)

# Split into two columns
cleaned_ids = pd.DataFrame(wc_data['PMID/PMCID'].values.tolist(), columns=['PMID','PMCID'])

# Get rid of decimals
cleaned_ids['PMID'] = cleaned_ids['PMID'].apply(lambda x: re.sub(r'(\..+)', '', str(x)))
cleaned_ids['PMCID'] = cleaned_ids['PMCID'].apply(lambda x: re.sub(r'(\..+)', '', str(x)))

# Get rid of invalid entries
cleaned_ids['PMID'] = cleaned_ids['PMID'].apply(lambda x: str(x).replace(x,'99999') if len(x) != 8 else x)
cleaned_ids['PMCID'] = cleaned_ids['PMCID'].apply(lambda x: str(x).replace(x,'99999') if len(x) != 7 else x)

# Replace 99999s with NaNs
cleaned_ids['PMID'] = cleaned_ids['PMID'].apply(lambda x: np.nan if x == '99999' else x)
cleaned_ids['PMCID'] = cleaned_ids['PMCID'].apply(lambda x: np.nan if x == '99999' else x)

In [5]:
# Create new dataframe for cleaned data
wc_data_clean = pd.DataFrame()
wc_data_clean['PMID'] = cleaned_ids['PMID']
wc_data_clean['PMCID'] = cleaned_ids['PMCID']
wc_data_clean['Publisher'] = wc_data['Publisher']
wc_data_clean['Journal'] = wc_data['Journal title']
wc_data_clean['Title'] = wc_data['Article title']
wc_data_clean['Cost'] = wc_data['COST (£) charged to Wellcome (inc VAT when charged)']

In [6]:
# Clean Cost
wc_data_clean['Cost'] = wc_data_clean['Cost'].apply(lambda x: re.sub('[^\.0-9 ]','', str(x)))
wc_data_clean['Cost'] = wc_data_clean['Cost'].apply(lambda x: float(x))
wc_data_clean['Cost'] = wc_data_clean['Cost'].apply(lambda x: np.nan if x >= 99999.00 else x)

In [7]:
# Strip whitespace
wc_data_clean['Publisher'] = wc_data_clean['Publisher'].apply(lambda x: str(x).strip())

# Define a function that replaces matching records with corrected strings
def replace_pub(match_str, replacement):
    replace_pat = re.compile('.*{}.*'.format(match_str), flags=re.IGNORECASE)
    wc_data_clean.Publisher.replace(replace_pat, replacement, regex=True, inplace=True)
    return

# Define which records need to be replaced with corrected strings
replacement_definitions = {'american chemical society':'American Chemical Society',
                           'american public health association':'American Public Health Association',
                           'american psychiatric':'American Psychiatric Association Publishing',
                           'for biochemistry and molecular':'American Society for Biochemistry and Molecular Biology',
                           'american society for microbiology':'American Society for Microbiology',
                           'asbmb':'American Society for Biochemistry and Molecular Biology',
                           'american society of hamatology':'American Society of Hematology',
                           'american society of haematology':'American Society of Hematology',

                           'bentha.': 'Bentham Science Publishers',
                           'biomed': 'BioMed Central',
                           'bmc': 'BioMed Central',
                           'b.ophysical society':'The Biophysical Society',
                           'society for endocrinology':'BioScientifica',
                           'bioscientifica':'BioScientifica',
                           'boulevard':'Boulevard',                           
                           'bmj': 'BMJ',
                           'british medical journal': 'BMJ',
                           
                           'cadmus':'Cenveo',
                           'cambridge':'Cambridge University Press',
                           'cup':'Cambridge University Press',
                           'cenveo':'Cenveo',
                           'cold spring':'Cold Spring Harbor Laboratory Press',
                           'company':'The Company of Biologists Ltd',
                           'copyright clear':'Copyright Clearance Center',
                           
                           'darmouth':'Dartmouth Journal Services',
                           'dartmouth':'Dartmouth Journal Services',
                           
                           'else':'Elsevier',
                           'endocrine soc':'The Endocrine Society',
                           'endrocrine soc':'The Endocrine Society',
                           'european society of':'European Society of Endocrinology',
                           'american societies for experimental biology':'Federation of American Societies for Experimental Biology',
                           'american society of experimental biology':'Federation of American Societies for Experimental Biology',
                           'faseb':'Federation of American Societies for Experimental Biology',
    
                           'american societies of Experimental Biology':'Federation of the American Societies of Experimental Biology',
                           'frontiers':'Frontiers Media SA',
                           'future medicine':'Future Science Publishing',
                           'future science':'Future Science Publishing',                           

                           'hindawi':'Hindawi Publishing Corporation',                           

                           'institute of physics':'IOP Publishing',
                           'informa':'Informa Healthcare Communications',
                           'crystallography':'International Union of Crystallography',
                           'tuberculosis':'International Union Against Tuberculosis and Lung Disease',
                           'impact':'Impact Journals LLC',
                           
                           'journal of visualized experiments':'MyJove Corp.',
                           'jove':'MyJove Corp.',
                           'j med internet research':'JMIR Publications',
                           
                           'karger':'Karger Publishers',
                           
                           'landes':'Landes Bioscience',
                           
                           'macmillan':'Palgrave Macmillan',                           
                           'mary':'Mary Ann Liebert, Inc.',
                           'mit press':'MIT Press',
                           'MDPI':'Molecular Diversity Preservation International',                           
                           
                           'national academy':'National Academy of Sciences',
                           'pnas':'National Academy of Sciences',
                           'nature': 'Nature Publishing',  
                           'npg': 'Nature Publishing',       

                           'optical society of america':'OSA Publishing',
                           'oxford':'Oxford University Press',

                           'plos':'Public Library of Science',
                           'policy press':'Policy Press',
                           'biochem journal':'Portland Press',
                           'portland':'Portland Press',
                           'pubmed':'PubMed Central',
                           'public.service':'publicservice.co.uk Ltd',

                           'sheridan':'Sheridan Press',
                           'springer':'Springer Publishing',
                           
                           'royal society':'Royal Society Publishing',
                           'rsc':'Royal Society Publishing',
                           
                           'sage':'SAGE Publications Inc',
                           'pion':'SAGE Publications Inc',
                           'society for gen':'Society for General Microbiology',
                           'leukocyte':'Society for Leukocyte Biology',
                           'society of neuro sciences':'Society for Neuroscience',
                           'society of neuroscience':'Society for Neuroscience',
                           'society for neuroscience':'Society for Neuroscience',
                           'dermato-venereologica':'Society for the Publication of Acta Dermato-Venereologica',

                           't&f':'Taylor & Francis Publishing',                           
                           'coaction':'Taylor & Francis Publishing',                           
                           'taylor':'Taylor & Francis Publishing',                           
                           'transcript verlag':'Transcript Verlag',

                           'wiley':'Wiley Publishing Inc',
                           'wliey':'Wiley Publishing Inc',
                           'lww':'Wolters Kluwer',
                           'wolters kluwer':'Wolters Kluwer',


                            # Definitions that might be parts of other words go last
                           'ambsb':'American Society for Biochemistry and Molecular Biology',
                           'asm': 'American Society for Microbiology',
                           'acs': 'American Chemical Society',
                           'aphs':'American Publich Health Association',
                           'oup':'Oxford University Press',
                           'brill':'Brill Publishers'
                          }

# Iterate through replacement definitions and call the correction function
for key,val in replacement_definitions.items():
    replace_pub(key, val)

In [8]:
# Strip whitespace
wc_data_clean['Journal'] = wc_data_clean['Journal'].apply(lambda x: str(x).strip())

# Define a function that replaces matching records with corrected strings
def replace_journ(match_str, replacement):
    replace_pat = re.compile('.*{}.*'.format(match_str), flags=re.IGNORECASE)
    wc_data_clean.Journal.replace(replace_pat, replacement, regex=True, inplace=True)
    return

# Define which records need to be replaced with corrected strings
replacement_definitions = {'acs nano':'ACS Nano',
                           '(^age$)':'Age',
                           'aids behav': 'AIDS and Behavior',
                           'aids care':'AIDS Care',
                           'aids journal':'AIDS',
                           'aids uk':'AIDS',
                           'Biological Crystallography':'Acta Crystallographica Section D: Biological Crystallography',
                           '(.*acta.*)( d)':'Acta Crystallographica Section D: Biological Crystallography',
                           'acta f':'Acta Crystallographica Section F: Structural Biology and Crystallization Communications',
                           'acta neuropathol':'Acta Neuropathologica',
                           'acta physio':'Acta Physiologica',
                           'Alimentrary Pharmacology & Therapeutics':'Alimentary Pharmacology & Therapeutics',
                           'Am J Bioeth':'American Journal of Bioethics',
                           'am j trop med hyg':'American Journal of Tropical Medicine & Hygiene',
                           'american chemical society':'Journal of the American Chemical Society',
                           'american jnl epidemiology':'American Journal of Epidemiology',
                           'American Journal of Bioethics--Neuroscience':'American Journal of Bioethics Neuroscience',
                           'American Journal of medical genetics':'American Journal of Medical Genetics',
                           'anal bioanal chem':'Analytical and Bioanalytical Chemistry',
                           'angew':'Angewandte Chemie International Edition',
                           'ents and Chemotherapy':'Antimicrobial Agents and Chemotherapy',
                           'Redox Signaling':'Antioxidants & Redox Signaling',
                           '(.*arthritis.*)(therapy)':'Arthritis Research and Therapy',
                           '(.*arthritis.*)(rheumatism)':'Arthritis and Rheumatism',
                           
                           '(.*biochem.*)(trans)':'Biochemical Society Transactions',
                           '(.*bio.*)(.*open.*)':'Biology Open',
                           '(.*biol.*)(.*chem.*)':'The Journal of Biological Chemistry',
                           '(.*br.*)(.*of op.*)':'British Journal of Ophthalmology',
                           '(.*bmj.*)(.*open.*)':'BMJ Open',
                           '(.*bmc.*)(.*genomics.*)':'BMJ Genomics',
                           '(.*biological psychology.*)':'Biological Psychology',
                           '(.*bjp.*)':'British Journal of Pharmacology',
                           '(.*quality and safety.*)':'BMJ Quality and Safety',
                           '(.*behav.*)(.*research.*)(.*therapy.*)':'Behavioral Research & Therapy',
                           '(.*biinformatics.*)':'Bioinformatics',
                           '(.*biochem.*)(.*journal.*)':'The Biochemical Journal',
                           '(^blood.*)':'Blood Journal',
                           '(.*brt.*)(.*pharm.*)':'British Journal of Clinical Pharmacology',
                           '(.*br.*)(.*of pharm.*)':'British Journal of Pharmacology',
                           '(.*br.*)(.*clinical pharm.*)':'British Journal of Clinical Pharmacology',
                           '(.*br.*)(.*psychiatry.*)':'The British Journal of Psychiatry',
                           
                           '(.*chembio.*)':'ChemBioChem',
                           '(.*current bio.*)':'Current Biology',
                           '(.*curr bio.*)':'Current Biology',
                           '(.*child:.*)':'Child: Care, Health Development',
                           '(.*cell death.*)(.*disease.*)':'Cell Death & Disease',
                           '(.*cell death.*)(.*diff.*)':'Cell Death & Differentiation',
                           '(.*cell host.*)':'Cell Host & Microbe',
                           '(.*cerebral cortex.*)':'Cerebral Cortex',
                           '(.*clinic.*)(.*infectious.*)':'Clinical Infectious Diseases',
                           '(.*chemmedchem.*)':'ChemMedChem',
                           '(.*consc.*)(.*cog.*)':'Consciousness & Cognition',
                           '(.*curr.*)(.*neuro.*)':'Current Opinions in Neurobiology',
                           
                           '(.*world.*)(.*bio.*)':'Developing World Bioethics',
                           '(.*development science.*)':'Developmental Science',
                           '(.*embo.*)':'EMBO Journal',
                           '(.*epigen.*)':'Epigenetics',
                           '(.*eur.*)(.*immun.*)':'European Journal of Immunology',
                           
                           '(.*febs.*)(.*j.*)':'FEBS Journal',
                           '(.*febs.*)(.*letter.*)':'FEBS Letters',
                           '(^gene$)':'Gene',
                           '(^genetics$)':'Genetics',
                           
                           '(.*haematologica.*)':'Haematologica',
                           '(.*hep.*)(.*tology.*)':'Journal of Hepatology',
                           '(.*health.*)(.*place.*)':'Health and Place',
                           '(.*human.*)(.*mapping.*)':'Human Brain Mapping',
                           '(^health$)':'Health: An Interdisciplinary Journal for the Social Study of Health, Illness and Medicine',
                           '(.*hum.*)(.*mol.*)':'Human Molecular Genetics',
                           '(.*hum.*)(.*mutation.*)':'Human Mutation',
                           
                           'IJTLD':'International Journal of Tuberculosis and Lung Disease',
                           '(.*int.*)(.*epidem.*)':'International Journal of Epidemiology',
                           '(.*iny.*)(.*epidem.*)':'International Journal of Epidemiology',
                           '(.*int.*)(.*parasit.*)':'International Journal of Parasitology',
                           
                           '(.*cardio.*)(.*magn.*)':'Journal of Cardiovascular Magnetic Resonance',
                           '(.*j.*)(.*cell )(.*sci.*)':'Journal of Cell Science',
                           '(.*clin.*)(.*microb.*)':'Journal of Clinical Microbiology',
                           '(.*j immunol.*)':'Journal of Immunology',
                           '(.*the journal of immunology.*)':'Journal of Immunology',
                           '(.*j.*)(.*infect.*)(.*dis.*)':'Journal of Infectious Diseases',
                           '(.*j.*)(.*med.*)(.*chem.*)':'Journal of Medical Chemistry',
                           'J Mol Biol':'Journal of Molecular Biology',
                           '(.*j.*)(.*steroid.*)(.*chem.*)':'Journal of Steroid Biochemistry & Molecular Biology',
                           '(.*j.*)(.*allergy.*)(.*clinical.*)':'Journal of Allergy and Clinical Immunology',
                           '(.*j.*)(.*bact.*)':'Journal of Bacteriology',
                           '(.*j.*)(.*neurochem.*)':'Journal of Neurochemistry',
                           '(.*j.*)(.*clinic.*)(.*endo.*)':'Journal of Clinical Endicronology and Metabolism',
                           '(.*j.*)(.*acq.*)(.*synd.*)':'Journal of Acquired Immune Deficiency Syndromes',
                           '(.*j.*)(.*autism.*)(.*dev.*)':'Journal of Autism and Developmental Disorders',
                           '(.*j.*)(.*behav.*)(.*exp.*)':'Journal of Behavior Therapy and Experimental Psychiatry',
                           '(.*epid.*)(.*comm.*)':'Journal of Epidemiology and Community Health',
                           '(.*neurosurg.*)(.*psychiatry.*)':'Journal of Neurology, Neurosurgery and Psychiatry',
                           'Journal of Virol':'Journal of Virology',
                           '(.*vis.*)(.*exp.*)':'Journal of Visualized Experiments',
                           '(.*cy.*)(.*fibr.*)':'Journal of Cystic Fibrosis',
                           '(.*cer.*)(.*met.*)':'Journal of Cerebral Blood Flow and Metabolism',
                           '(.*journal of.*)(.*mol.*)(.*endo)':'Molecular Endocrinology',
                           'Molecular Endocrinology':'Journal of Molecular Endocrinology',
                           '(.*mol.*)(.*cellular.*)(.*endo.*)':'Molecular and Cellular Endocrinology',
                           '(.*roy.*)(.*inter.*)':'Journal of the Royal Society Interface',
                           
                           '(^Lancet$)':'The Lancet',
                           '(^molecular biology$)':'Molecular Biology',
                           '(.*molec.*)(.*proteomics.*)':'Molecular and Cellular Proteomics',
                           '(.*child.*)(.*nutr.*)':'Maternal and Child Nutrition',
                           '(.*microbe.*)(.*infect.*)':'Microbes and Infection',
                           
                           '(.*nature.*)(.*comm.*)':'Nature Communications',
                           '(.*neph.*)(.*dial.*)':'Nephrology, Dialysis, and Transplantation',
                           '(.*nucleic.*)':'Nucleic Acids Research',
                           
                           '(.*plos.*)(.*one.*)':'Public Library of Science ONE',
                           '(.*plos.*)(.*1.*)':'Public Library of Science ONE',
                           '(.*plos.*)(.*comp.*)':'Public Library of Science Computational Biology',
                           '(.*plos.*)(.*gene.*)':'Public Library of Science Genetics',
                           '(.*plos.*)(.*tropical.*)':'Public Library of Science Neglected Tropical Diseases',
                           '(.*plos.*)(.*ntd.*)':'Public Library of Science Neglected Tropical Diseases',
                           '(.*plos.*)(.*path.*)':'Public Library of Science Pathogens',
                           '(.*plos.*)(.*bio.*)':'Public Library of Science Biology',
                           '(.*plos.*)(.*med.*)':'Public Library of Science Medicine',
                           '(.*plos.*)':'Public Library of Science',
                           '(.*pnas.*)':'Proceedings of the National Academy of Sciences',
                           '(.*proc.*)(.*nat.*)':'Proceedings of the National Academy of Sciences',
                           'proceedings of royal society':'Proceedings of the Royal Society',
                           '(.*proceedings.*)(.b.*)':'Proceedings of the Royal Society B: Biological Sciences',
                           '(.*philo.*)(.b.*)':'Philosophical Transactions of the Royal Society of London. Series B, Biological Sciences',
                           'Parsitology':'Parasitology',
                           '(.*par.*)(.*vector.*)':'Parasites and Vectors',
                           '(.*pflugers.*)':'Pflugers Archive',
                           
                           '(^rna.*)(.*biology.*)':'RNA Biology',
                           
                           '(.*sex.*)(.*trans.*)(.*inf.*)':'Sexually Transmitted Infections',
                           '(.*soc.*)(.*epid.*)':'Social Psychiatry and Psychiatric Epidemiology',
                           '(.*social science.*)(.*med.*)':'Social Science and Medicine',
                           
                           '(.*am.*)(.*j.*)(.*hum.*)(.*gen.*)':'American Journal of Human Genetics'

                          }


# Iterate through replacement definitions and call the correction function
for key,val in replacement_definitions.items():
    replace_journ(key, val)
    
# Strip 'the' from the beginning of titles
wc_data_clean['Journal'] = wc_data_clean['Journal'].apply(lambda x: re.sub("(^the )", "", x, flags=re.IGNORECASE))

In [25]:
# Determine the five most common journals and the total articles for each.
# Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal .

top_journals = wc_data_clean.groupby('Journal').agg({'Cost': ['count','mean', 'median','std']})
top_journals = top_journals.reset_index()
top_journals = top_journals.sort_values(by=('Cost','count'), ascending=False)

print(top_journals.head(n=5))

                                             Journal  Cost               \
                                                     count         mean   
654                    Public Library of Science ONE   199   935.577236   
410                  Journal of Biological Chemistry    71  1378.140423   
633  Proceedings of the National Academy of Sciences    36   772.919167   
599                           Nucleic Acids Research    29  1162.344828   
657              Public Library of Science Pathogens    22  1572.866818   

                          
      median         std  
654   896.99  194.654385  
410  1311.73  388.553911  
633   732.01  370.191932  
599   852.00  442.150934  
657  1600.25  161.780891  
